<a href="https://colab.research.google.com/github/POTTERONTHEHOUSE/1a/blob/master/GIT1aM128L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!pip3 install Cython #this does solve the "NameError: name '_C' is not defined" problem
import os
import math
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd import Variable
from collections import OrderedDict
import torch.nn.functional as F
# Hyper Parameters
EPOCH =120         # never mind i got early stop

BATCH_SIZE =128    
LR = 0.001              # learning rate 
M=128
L=2
result=torch.tensor([0,0,0,0])# the output of the this program：[M,L,training accuracy, test accuaracy]
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=True, delta=0):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score - self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:#so if score < self.best_score 7 times in a row, then self.early_stop = True.
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when test loss decrease.'''
        if self.verbose:
            print(f'test error decreased ({self.val_loss_min:.2f} --> {val_loss:.2f}).  Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss
#torch.manual_seed(1)    # reproducible

# prepare data for CNN
transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_data = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,                                    
    transform=transform,   

    download=True
)

# Data Loader for easy mini-batch return in training
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE , shuffle=True)


test_data = torchvision.datasets.CIFAR10(root='./data', train=False,download=True,transform=transform)
testloader = Data.DataLoader(test_data, batch_size=BATCH_SIZE , shuffle=False)


#for the first conv layer,as is discribed in the paper
def _pool_function_factory(relu,conv):
    def pool_function(inputs):
      _output=pool(relu(conv(inputs)))
      return _output
    return pool_function
#define a basic 'layer' with relu, without batch normalization
def _bn_function_factory(conv,relu):
  def bn_function(inputs):
    _output=relu(conv(inputs))
    return _output
  return bn_function

#define a recursive layer which includes norm and relu
class _recursive(nn.Module):
  def __init__(self,M):
    super(_recursive,self).__init__()
    self.add_module('convr',nn.Conv2d(M,M,kernel_size=3,padding=1,bias=True))
    #self.add_module('normr',nn.BatchNorm2d(M))
    self.add_module('relur',nn.ReLU())
  
  def forward(self, inputs):
    #output=_bn_function_factory(self.convr,self.relur)
    output=self.convr(inputs)
    output=self.relur(output)
    return output

class CNN(nn.Module):
    def __init__(self,num_classes=10):
        super(CNN, self).__init__()
        #self.features1 = nn.Sequential(
            #1
            #nn.Conv2d(3,M,kernel_size=8,padding=4,bias=True),
            #nn.ReLU(),
            #nn.MaxPool2d(kernel_size=4,stride=4),
        #)
        #first layer
        self.features=nn.Sequential(OrderedDict([
                ('conv1', nn.Conv2d(3, M, kernel_size=8, padding=4, bias=True)),
            ]))
        '''self.conv1=nn.Conv2d(3, M, kernel_size=8, padding=4, bias=True),    
        self.features=self.conv1,#then features become tuple and can not be added modules
        #and it says 'tuple' object has no attribute 'size', strange why'''
        
        self.features.add_module('relu1',nn.ReLU()),
        
        self.features.add_module('pool1',nn.MaxPool2d(kernel_size=4,stride=4)),
        
        #2~L+1 layers,recursive
        for i in range(1,L+1):
          layer=_recursive(M)
          self.features.add_module('recursivelayer%d'%(i),layer)
        
        #After the last hidden layer
        self.classifier =  nn.Linear(M*8*8,num_classes)
          
        # Initialization
        for name, param in self.named_parameters():
            if 'conv' in name and 'weight' in name:
                param.data.normal_(0,0.1) 
            elif 'norm' in name and 'weight' in name:
                param.data.fill_(1)
            elif 'norm' in name and 'bias' in name:
                param.data.fill_(0)
            elif 'classifier' in name and 'bias' in name:
                param.data.fill_(0)

    def forward(self, x):
        out= self.features(x)
        #print('out1=',out)
        #print('out1size=',out.size())
        out = out.view(out.size(0), -1)
        out = self.classifier(out)#logistic classifier ;softmax is in nn.CrossEntropyLoss
        return out    

cnn = CNN()
print(cnn)
#initialization function
def weight_init(m):
    classname = m.__class__.__name__ # 得到网络层的名字，如ConvTranspose2d
    if classname.find('Conv') != -1:  # 使用了find函数，如果不存在返回值为-1，所以让其不等于-1
        m.weight.data.normal_(0.0, 0.1)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)#the paper did not mention the initialization of BN but whatever
        m.bias.data.fill_(0)


cnn.apply(weight_init)

optimizer = torch.optim.SGD(cnn.parameters(), lr=0.001, momentum=0.9)   
# The L2 regularization on the parameters of the model is already included in most optimizers, including optim.SGD
loss_func = nn.CrossEntropyLoss()                       

# initialize the early_stopping object
early_stopping = EarlyStopping()


# training and testing
for epoch in range(EPOCH):
  run_correct=0 
  train_accuracy=0
  running_loss = 0.0
    
  for step, (b_x, b_y) in enumerate(train_loader):   # gives batch data, normalize x when iterate train_loader
            
      b_x=Variable(b_x,requires_grad=True)

      output = cnn(b_x)            # cnn output
      _,predictrain=torch.max(output,1)
      num_correct=(predictrain==b_y).sum()
      run_correct+=num_correct.item()

      loss = loss_func(output, b_y)   # cross entropy loss
      optimizer.zero_grad()           # clear gradients for this training step
      loss.backward()                 # backpropagation, compute gradients
      optimizer.step()                # apply gradients
      # print statistics
  train_accuracy=100*run_correct / (len(train_data))
  print('Finish {0} epoch, training accuracy: {1:.2f}%--L={2},M={3}'.format(epoch + 1, train_accuracy,L,M))
      
      

  total_correct = 0
  total_images = 0
  test_losses=[]
  test_loss=0
  avg_test_losses=[]
  with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = cnn(images)
            outloss=loss_func(outputs,labels)
            test_losses.append(outloss.item())
            _, predicted = torch.max(outputs.data, 1)
            total_images += labels.size(0)
            total_correct += (predicted == labels).sum().item()
        test_loss = np.average(test_losses)  
        avg_test_losses.append(test_loss)  
        test_losses = []

  test_accuracy = total_correct / total_images * 100
  print('In Epoch {0}, testing accuracy: {1:.2f}% --L={2},M={3}'.format(epoch+1,test_accuracy,L,M))

  # load the last checkpoint with the best model for the early stop
  early_stopping(test_loss, cnn)#has save model inside
  if early_stopping.early_stop:
      print("Early stopping")
      break
  # load the last checkpoint with the best model for the early stop

result=[M,L,train_accuracy,test_accuracy]
torch.save(result,F"/content/gdrive/My Drive/{'resultM128L2.pt'}" )
print(result)
   




Files already downloaded and verified
Files already downloaded and verified
CNN(
  (features): Sequential(
    (conv1): Conv2d(3, 128, kernel_size=(8, 8), stride=(1, 1), padding=(4, 4))
    (relu1): ReLU()
    (pool1): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    (recursivelayer1): _recursive(
      (convr): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (relur): ReLU()
    )
    (recursivelayer2): _recursive(
      (convr): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (relur): ReLU()
    )
  )
  (classifier): Linear(in_features=8192, out_features=10, bias=True)
)
Finish 1 epoch, training accuracy: 46.09%--L=2,M=128
In Epoch 1, testing accuracy: 53.58% --L=2,M=128
test error decreased (inf --> 1.31).  Saving model ...
Finish 2 epoch, training accuracy: 58.71%--L=2,M=128
In Epoch 2, testing accuracy: 58.12% --L=2,M=128
test error decreased (1.31 --> 1.18).  Saving model ...
Finish 3 epoch, training